In [202]:
def get_line_number(file_path):
    sys.stderr.write("Counting line number of {}".format(file_path))
    f = gzip.open(file_path)
    lines = f.readlines()
    count =0
    
    for line in lines:
        count += 1
    return count

def parse(filename, total):
    IGNORE_FIELDS = ['Total items', 'reviews']
    f = open(filename, 'r')
    lines = f.readlines()
    entry = {}
    categories = []
    reviews = []
    similar_items = []
  
    for line in lines[0:total]:
        colonPos = line.find(':')

        if line.startswith("Id"):
            if reviews:
                entry["reviews"] = reviews
            if categories:
                entry["categories"] = categories
            yield entry
            entry = {}
            categories = []
            reviews = []
            rest = line[colonPos+2:]
            entry["id"] = rest[1:-1]
      
        elif line.startswith("similar"):
            similar_items = line.split()[2:]
            entry['similar_items'] = similar_items

    # "cutomer" is typo of "customer" in original data
        elif line.find("cutomer:") != -1:
            review_info = line.split()
            reviews.append({'customer_id': review_info[2], 
                          'rating': int(review_info[4]), 
                          'votes': int(review_info[6]), 
                          'helpful': int(review_info[8])})

        elif line.startswith("   |"):
            categories.append(line[0:-1].replace(' ',''))

        elif colonPos != -1:
            eName = line[:colonPos]
            rest = line[colonPos+2:]
            if not eName in IGNORE_FIELDS:
                if eName[0] == ' ':
                    eName = eName[2:]
                entry[eName] = rest[0:-1].replace("'","''")

    if reviews:
        entry["reviews"] = reviews
    if categories:
        entry["categories"] = categories
    
    yield entry

In [203]:
file_path = "data/amazon-meta.txt"
line_num = sum(1 for line in open(file_path))
result = []
print(line_num)
for e in parse(file_path, total=882):
    if e:
        result.append(e)
print(result[0].keys())

15010574
dict_keys(['id', 'ASIN'])


In [204]:
import psycopg2
necessaryKeys = ["id","ASIN","title","group","salesrank","similar"]

con = psycopg2.connect( user = "postgres",
                        password = "postgres",
						host = "localhost",
						port = "5432",
						dbname = "amazon")
cursor = con.cursor()
cursor.execute('DELETE FROM similars;DELETE FROM products')
con.commit()
# -------------------------------------
for value in result:
    for key in necessaryKeys:
        if key not in list(value.keys()):
            value[key] = "null"
        else:
            if key != 'id' and key != 'salesrank' and key !='similar':
                value[key] = "'{}'".format(value[key])
    query = 'INSERT INTO products VALUES ({},{},{},{},{})'.format(int(value['id']),value['ASIN'],value['title'],value['group'],value['salesrank'])
    cursor.execute(query)

    query = 'INSERT INTO similars VALUES '
    similars = value['similar'][1:].split('  ')[1:]
    if similars:
        for similar in similars:
            query = query + '({},\'{}\'),'.format(value['ASIN'],similar)
        query = query[:-1]
        cursor.execute(query)
# -------------------------------------

con.commit()
cursor.close()
con.close()